In [ ]:
# 该文件用于删除class-person文件夹下冗余的图像
import os
import shutil
from collections import defaultdict

# 定义目标目录
target_dir = "/home/humw/Codes/My-Anti-DreamBooth/data/class-person"

# 获取目标目录下的所有文件
files = os.listdir(target_dir)

# 使用defaultdict来分组以数字n开头的文件
file_groups = defaultdict(list)

# 遍历文件并分组
for file in files:
    if file.endswith(".jpg") and "-" in file:
        # 拆分文件名，获取数字n部分
        num_part = file.split("-")[0]
        file_groups[num_part].append(file)

# 遍历每个分组，只保留一个文件
for num, file_list in file_groups.items():
    if len(file_list) > 1:
        # 选择保留第一个文件，并删除其他文件
        file_to_keep = file_list[0]
        for file in file_list[1:]:
            file_path = os.path.join(target_dir, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

print("Process completed.")

In [ ]:
import os

print(len(os.listdir("/home/humw/Codes/T2VSafetyBench/My-Anti-DreamBooth/data/class-person")))

In [ ]:
# 检测图像阈值，MetaCloak模式
from torchvision import transforms
import torch
from pathlib import Path
from PIL import Image

def load_data(data_dir, size=512, center_crop=True) -> torch.Tensor:
    image_transforms = transforms.Compose(
        [
            transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(size) if center_crop else transforms.RandomCrop(size),
            transforms.ToTensor(),
            # transforms.Normalize([0.5], [0.5]),
        ]
    )

    images = [image_transforms(Image.open(i).convert("RGB")) for i in sorted(list(Path(data_dir).iterdir()))]
    images = torch.stack(images)
    return images

weight_type = torch.bfloat16
clean_leaf_id_pixel_values = load_data('/home/humw/Datasets/VGGFace2/n000050/set_B').to(dtype=weight_type)
adv_leaf_id_pixel_values = load_data('/home/humw/Codes/My-Anti-DreamBooth/outputs/adversarial_images/test_ipadapter_sd1-5/n000050').to(dtype=weight_type)
et = adv_leaf_id_pixel_values - clean_leaf_id_pixel_values

et = abs(et)
print("11/255:{}".format(11/255))
print("et min:{}".format(et.min()))
print("et max:{}".format(et.max()))
print("et mean:{}".format(et.mean()))
et = et.reshape(-1)
a = et
cnt1 = 0
cnt2 = 0
for t in a:
    if t > 11/255:
        cnt1 = cnt1 + 1
    if t > 22/255:
        cnt2 = cnt2 + 1
print("proportion of pixels larger than 11/255:{}".format(cnt1/et.shape[0]))
print("proportion of pixels larger than 22/255:{}".format(cnt2/et.shape[0]))

In [ ]:
16/255

In [ ]:
# 打印所有的person ID
import os

path = "/home/humw/Datasets/VGGFace2"

print(sorted(os.listdir(path)))